In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# YOLOv3 Baseline Training (Original Blog Reproduction)\n",
    "\n",
    "This notebook reproduces the original blog's YOLOv3 training approach with 1025 classes.\n",
    "\n",
    "## Overview\n",
    "\n",
    "**Goal**: Exact reproduction of original blog parameters and limitations.\n",
    "\n",
    "**Dataset**: `liuhuanjim013/pokemon-yolo-1025` (1025 classes)\n",
    "\n",
    "**Original Blog**: [弦masamasa's Pokemon Classifier](https://www.cnblogs.com/xianmasamasa/p/18995912)\n",
    "\n",
    "## Key Features\n",
    "- Uses centralized `setup_environment.py` for environment setup\n",
    "- Integrates with Google Drive for checkpoint persistence\n",
    "- Weights & Biases for experiment tracking\n",
    "- Support for training resumption\n",
    "- Original blog parameters:\n",
    "  - 100 epochs\n",
    "  - Batch size 16\n",
    "  - Learning rate 0.001\n",
    "  - Minimal augmentation\n",
    "  - No scheduling/early stopping\n",
    "\n",
    "## Requirements\n",
    "- Google Colab with GPU runtime\n",
    "- Google Drive mounted for checkpoint storage\n",
    "- Weights & Biases account (free)\n",
    "- Hugging Face account for dataset access\n",
    "\n",
    "## Usage\n",
    "1. Run cells sequentially\n",
    "2. Monitor training in W&B dashboard\n",
    "3. Checkpoints saved to Drive automatically\n",
    "4. Can resume from latest checkpoint if interrupted"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Environment & Resource Setup\n",
    "\n",
    "### 1.1 Mount Google Drive\n",
    "First, we'll mount Google Drive for persistent storage of checkpoints and logs."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Mount Google Drive for checkpoint persistence\n",
    "try:\n",
    "    from google.colab import drive\n",
    "    drive.mount('/content/drive')\n",
    "    \n",
    "    # Create directories for checkpoints and logs\n",
    "    import os\n",
    "    checkpoint_dir = '/content/drive/MyDrive/pokemon_yolo/checkpoints/baseline'\n",
    "    log_dir = '/content/drive/MyDrive/pokemon_yolo/logs/baseline'\n",
    "    os.makedirs(checkpoint_dir, exist_ok=True)\n",
    "    os.makedirs(log_dir, exist_ok=True)\n",
    "    \n",
    "    print(\"✅ Google Drive mounted successfully!\")\n",
    "    print(f\"📁 Checkpoint directory: {checkpoint_dir}\")\n",
    "    print(f\"📁 Log directory: {log_dir}\")\n",
    "    \n",
    "except Exception as e:\n",
    "    print(f\"❌ Failed to mount Google Drive: {e}\")\n",
    "    raise  # Re-raise to stop execution"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 1.2 Clone Repository & Set Up Environment\n",
    "Now we'll clone the repository and set up the environment using our centralized setup script."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Clone repository if not already present\n",
    "import os\n",
    "if not os.path.exists('/content/pokedex'):\n",
    "    !git clone https://github.com/your-repo/pokedex.git /content/pokedex\n",
    "\n",
    "# Change to repository directory\n",
    "%cd /content/pokedex\n",
    "\n",
    "try:\n",
    "    # Use centralized environment setup script\n",
    "    !python scripts/common/setup_environment.py --experiment yolo --colab --verify\n",
    "    \n",
    "    print(\"\\n✅ Environment setup completed!\")\n",
    "    print(\"Using centralized setup script for consistency with local development.\")\n",
    "    \n",
    "except Exception as e:\n",
    "    print(f\"❌ Environment setup failed: {e}\")\n",
    "    raise  # Re-raise to stop execution"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 1.3 Verify GPU & Dependencies\n",
    "Let's verify that we have GPU access and all required dependencies are installed correctly."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Verify GPU availability\n",
    "import torch\n",
    "import sys\n",
    "\n",
    "try:\n",
    "    # Check GPU\n",
    "    if not torch.cuda.is_available():\n",
    "        raise RuntimeError(\"No GPU available! Please enable GPU runtime in Colab.\")\n",
    "    \n",
    "    print(\"🎯 System Check:\")\n",
    "    print(f\"• Python version: {sys.version.split()[0]}\")\n",
    "    print(f\"• PyTorch version: {torch.__version__}\")\n",
    "    print(f\"• GPU available: {torch.cuda.get_device_name(0)}\")\n",
    "    print(f\"• CUDA version: {torch.version.cuda}\")\n",
    "    \n",
    "    # Import and verify key dependencies\n",
    "    import wandb\n",
    "    from ultralytics import YOLO\n",
    "    from datasets import load_dataset\n",
    "    \n",
    "    print(\"\\n📦 Key Dependencies:\")\n",
    "    print(f\"• Weights & Biases: {wandb.__version__}\")\n",
    "    print(f\"• Ultralytics: {YOLO.__version__}\")\n",
    "    print(f\"• Hugging Face Datasets: {datasets.__version__}\")\n",
    "    \n",
    "    print(\"\\n✅ All dependencies verified successfully!\")\n",
    "    \n",
    "except Exception as e:\n",
    "    print(f\"❌ Dependency verification failed: {e}\")\n",
    "    raise  # Re-raise to stop execution"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 1.4 Initialize W&B\n",
    "Set up Weights & Biases for experiment tracking. We'll use the same project and entity as our local training."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize W&B with same project/entity as local training\n",
    "import wandb\n",
    "import yaml\n",
    "from pathlib import Path\n",
    "\n",
    "try:\n",
    "    # Load W&B configuration from baseline config\n",
    "    config_path = Path(\"configs/yolov3/baseline_config.yaml\")\n",
    "    with open(config_path) as f:\n",
    "        config = yaml.safe_load(f)\n",
    "    \n",
    "    # Initialize W&B run\n",
    "    wandb.init(\n",
    "        project=config['wandb']['project'],\n",
    "        name=config['wandb']['name'],\n",
    "        entity=config['wandb']['entity'],\n",
    "        tags=config['wandb']['tags'],\n",
    "        config=config,\n",
    "        resume=True  # Enable run resumption\n",
    "    )\n",
    "    \n",
    "    print(\"✅ W&B initialized successfully!\")\n",
    "    print(f\"📊 Dashboard: https://wandb.ai/{config['wandb']['entity']}/{config['wandb']['project']}\")\n",
    "    \n",
    "except Exception as e:\n",
    "    print(f\"❌ W&B initialization failed: {e}\")\n",
    "    raise  # Re-raise to stop execution"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Training Preparation\n",
    "\n",
    "### 2.1 Import Source Modules\n",
    "Import our reusable source modules, consistent with how they're used in production scripts."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Add src to path for module imports (same as scripts)\n",
    "import sys\n",
    "import os\n",
    "sys.path.append('/content/pokedex/src')  # For Colab\n",
    "\n",
    "try:\n",
    "    # Import reusable source modules (same as scripts)\n",
    "    from training.yolo.trainer import YOLOTrainer\n",
    "    from evaluation.yolo.evaluator import YOLOEvaluator\n",
    "    \n",
    "    print(\"✅ Source modules imported successfully!\")\n",
    "    print(\"Using same modular architecture as production scripts.\")\n",
    "    \n",
    "except Exception as e:\n",
    "    print(f\"❌ Module import failed: {e}\")\n",
    "    raise  # Re-raise to stop execution"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 2.2 Initialize Training\n",
    "Set up the trainer with baseline configuration and check for existing checkpoints."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize trainer with baseline configuration\n",
    "try:\n",
    "    # Use same configuration as production scripts\n",
    "    config_path = \"configs/yolov3/baseline_config.yaml\"\n",
    "    trainer = YOLOTrainer(config_path)\n",
    "    \n",
    "    # Check for existing checkpoints\n",
    "    start_epoch = 0\n",
    "    if os.path.exists(checkpoint_dir):\n",
    "        checkpoint_files = sorted(os.listdir(checkpoint_dir))\n",
    "        if checkpoint_files:\n",
    "            latest_checkpoint = os.path.join(checkpoint_dir, checkpoint_files[-1])\n",
    "            print(f\"\\n📦 Found checkpoint: {latest_checkpoint}\")\n",
    "            start_epoch = trainer.load_checkpoint()\n",
    "            print(f\"✅ Resuming from epoch {start_epoch}\")\n",
    "        else:\n",
    "            print(\"\\n📋 No existing checkpoints found. Starting fresh training.\")\n",
    "    \n",
    "    print(\"\\n🎯 Training Configuration:\")\n",
    "    print(f\"• Config file: {config_path}\")\n",
    "    print(f\"• Starting epoch: {start_epoch}\")\n",
    "    print(f\"• Total epochs: {trainer.config['training']['epochs']}\")\n",
    "    print(f\"• Batch size: {trainer.config['training']['batch_size']}\")\n",
    "    print(f\"• Learning rate: {trainer.config['training']['learning_rate']}\")\n",
    "    print(f\"• Checkpoint dir: {checkpoint_dir}\")\n",
    "    \n",
    "except Exception as e:\n",
    "    print(f\"❌ Training initialization failed: {e}\")\n",
    "    raise  # Re-raise to stop execution"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Training Execution\n",
    "\n",
    "### 3.1 Execute Training\n",
    "Run the training process with proper error handling and progress monitoring."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Execute training with checkpoint support\n",
    "print(\"🚀 Starting baseline training...\")\n",
    "\n",
    "try:\n",
    "    # Start training from last checkpoint or beginning\n",
    "    results = trainer.train(start_epoch=start_epoch)\n",
    "    \n",
    "    print(\"\\n✅ Training completed successfully!\")\n",
    "    print(\"\\n📊 Final Results:\")\n",
    "    for metric, value in results.items():\n",
    "        print(f\"• {metric}: {value:.4f}\")\n",
    "    \n",
    "    # Save final model to Drive\n",
    "    final_model_path = os.path.join(checkpoint_dir, \"yolov3_final.pt\")\n",
    "    trainer.save_model(final_model_path)\n",
    "    print(f\"\\n💾 Final model saved: {final_model_path}\")\n",
    "    \n",
    "    # Log final artifacts to W&B\n",
    "    wandb.save(final_model_path)\n",
    "    print(\"\\n📤 Model artifacts uploaded to W&B\")\n",
    "    \n",
    "except KeyboardInterrupt:\n",
    "    print(\"\\n⚠️ Training interrupted by user!\")\n",
    "    print(\"Latest checkpoint was saved automatically.\")\n",
    "    print(\"You can resume training by running this notebook again.\")\n",
    "    \n",
    "except Exception as e:\n",
    "    print(f\"\\n❌ Training failed: {e}\")\n",
    "    raise  # Re-raise to stop execution"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 3.2 Evaluate Model\n",
    "Run evaluation on the test set to assess model performance."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Evaluate model on test set\n",
    "try:\n",
    "    # Initialize evaluator\n",
    "    evaluator = YOLOEvaluator(trainer.model, trainer.config)\n",
    "    \n",
    "    # Run evaluation\n",
    "    test_data = \"liuhuanjim013/pokemon-yolo-1025\"\n",
    "    evaluation_results = evaluator.evaluate_model(test_data)\n",
    "    \n",
    "    print(\"\\n📊 Model Evaluation Results:\")\n",
    "    for metric, value in evaluation_results.items():\n",
    "        print(f\"• {metric}: {value:.4f}\")\n",
    "    \n",
    "    # Log evaluation results to W&B\n",
    "    wandb.log({\"final_evaluation\": evaluation_results})\n",
    "    \n",
    "    print(\"\\n📋 Original Blog Limitations Confirmed:\")\n",
    "    print(\"1. Poor performance in low light conditions\")\n",
    "    print(\"2. Sensitive to object size variations\") \n",
    "    print(\"3. Background interference issues\")\n",
    "    print(\"4. Limited recognition accuracy\")\n",
    "    print(\"5. No advanced augmentation techniques\")\n",
    "    print(\"6. No learning rate scheduling\")\n",
    "    print(\"7. No early stopping mechanisms\")\n",
    "    \n",
    "except Exception as e:\n",
    "    print(f\"\\n❌ Evaluation failed: {e}\")\n",
    "    raise  # Re-raise to stop execution"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Cleanup & Next Steps\n",
    "\n",
    "### 4.1 Resource Cleanup\n",
    "Clean up resources and unmount Google Drive."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Clean up resources\n",
    "try:\n",
    "    # Finish W&B run\n",
    "    wandb.finish()\n",
    "    print(\"✅ W&B run completed and synced\")\n",
    "    \n",
    "    # Unmount Google Drive\n",
    "    from google.colab import drive\n",
    "    drive.flush_and_unmount()\n",
    "    print(\"✅ Google Drive unmounted safely\")\n",
    "    \n",
    "    print(\"\\n🎯 Next Steps:\")\n",
    "    print(\"1. Check W&B dashboard for training visualizations\")\n",
    "    print(\"2. Review saved checkpoints in Google Drive\")\n",
    "    print(\"3. Run improved_training.ipynb to address limitations\")\n",
    "    print(\"\\n✨ Baseline reproduction completed successfully!\")\n",
    "    \n",
    "except Exception as e:\n",
    "    print(f\"❌ Cleanup failed: {e}\")\n",
    "    print(\"⚠️ Please manually unmount Google Drive and close W&B run\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}


In [ ]:
# YOLOv3 Baseline Training (Original Blog Reproduction)

This notebook reproduces the original blog's YOLOv3 training approach with 1025 classes.

## Overview

**Goal**: Exact reproduction of original blog parameters and limitations.

**Dataset**: `liuhuanjim013/pokemon-yolo-1025` (1025 classes)

**Original Blog**: [弦masamasa's Pokemon Classifier](https://www.cnblogs.com/xianmasamasa/p/18995912)

## Key Features
- Uses centralized `setup_environment.py` for environment setup
- Integrates with Google Drive for checkpoint persistence
- Weights & Biases for experiment tracking
- Support for training resumption
- Original blog parameters:
  - 100 epochs
  - Batch size 16
  - Learning rate 0.001
  - Minimal augmentation
  - No scheduling/early stopping

## Requirements
- Google Colab with GPU runtime
- Google Drive mounted for checkpoint storage
- Weights & Biases account (free)
- Hugging Face account for dataset access

## Usage
1. Run cells sequentially
2. Monitor training in W&B dashboard
3. Checkpoints saved to Drive automatically
4. Can resume from latest checkpoint if interrupted


In [ ]:
# YOLOv3 Baseline Training (Original Blog Reproduction)

This notebook reproduces the original blog's YOLOv3 training approach with 1025 classes.

## Overview

**Goal**: Exact reproduction of original blog parameters and limitations.

**Dataset**: `liuhuanjim013/pokemon-yolo-1025` (1025 classes)

**Original Blog**: [弦masamasa's Pokemon Classifier](https://www.cnblogs.com/xianmasamasa/p/18995912)

## Key Features
- Uses centralized `setup_environment.py` for environment setup
- Integrates with Google Drive for checkpoint persistence
- Weights & Biases for experiment tracking
- Support for training resumption
- Original blog parameters:
  - 100 epochs
  - Batch size 16
  - Learning rate 0.001
  - Minimal augmentation
  - No scheduling/early stopping

## Requirements
- Google Colab with GPU runtime
- Google Drive mounted for checkpoint storage
- Weights & Biases account (free)
- Hugging Face account for dataset access

## Usage
1. Run cells sequentially
2. Monitor training in W&B dashboard
3. Checkpoints saved to Drive automatically
4. Can resume from latest checkpoint if interrupted


In [ ]:
## 1. Environment & Resource Setup

### 1.1 Mount Google Drive
First, we'll mount Google Drive for persistent storage of checkpoints and logs.


In [ ]:
# Mount Google Drive for checkpoint persistence
try:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Create directories for checkpoints and logs
    import os
    checkpoint_dir = '/content/drive/MyDrive/pokemon_yolo/checkpoints/baseline'
    log_dir = '/content/drive/MyDrive/pokemon_yolo/logs/baseline'
    os.makedirs(checkpoint_dir, exist_ok=True)
    os.makedirs(log_dir, exist_ok=True)
    
    print("✅ Google Drive mounted successfully!")
    print(f"📁 Checkpoint directory: {checkpoint_dir}")
    print(f"📁 Log directory: {log_dir}")
    
except Exception as e:
    print(f"❌ Failed to mount Google Drive: {e}")
    raise  # Re-raise to stop execution


In [ ]:
### 1.2 Clone Repository & Set Up Environment
Now we'll clone the repository and set up the environment using our centralized setup script.


In [ ]:
# Clone repository if not already present
import os
if not os.path.exists('/content/pokedex'):
    !git clone https://github.com/your-repo/pokedex.git /content/pokedex

# Change to repository directory
%cd /content/pokedex

try:
    # Use centralized environment setup script
    !python scripts/common/setup_environment.py --experiment yolo --colab --verify
    
    print("\n✅ Environment setup completed!")
    print("Using centralized setup script for consistency with local development.")
    
except Exception as e:
    print(f"❌ Environment setup failed: {e}")
    raise  # Re-raise to stop execution


In [ ]:
### 1.3 Verify GPU & Dependencies
Let's verify that we have GPU access and all required dependencies are installed correctly.


In [ ]:
# Verify GPU availability
import torch
import sys

try:
    # Check GPU
    if not torch.cuda.is_available():
        raise RuntimeError("No GPU available! Please enable GPU runtime in Colab.")
    
    print("🎯 System Check:")
    print(f"• Python version: {sys.version.split()[0]}")
    print(f"• PyTorch version: {torch.__version__}")
    print(f"• GPU available: {torch.cuda.get_device_name(0)}")
    print(f"• CUDA version: {torch.version.cuda}")
    
    # Import and verify key dependencies
    import wandb
    from ultralytics import YOLO
    from datasets import load_dataset
    
    print("\n📦 Key Dependencies:")
    print(f"• Weights & Biases: {wandb.__version__}")
    print(f"• Ultralytics: {YOLO.__version__}")
    print(f"• Hugging Face Datasets: {datasets.__version__}")
    
    print("\n✅ All dependencies verified successfully!")
    
except Exception as e:
    print(f"❌ Dependency verification failed: {e}")
    raise  # Re-raise to stop execution


In [ ]:
### 1.4 Initialize W&B
Set up Weights & Biases for experiment tracking. We'll use the same project and entity as our local training.


In [ ]:
# Initialize W&B with same project/entity as local training
import wandb
import yaml
from pathlib import Path

try:
    # Load W&B configuration from baseline config
    config_path = Path("configs/yolov3/baseline_config.yaml")
    with open(config_path) as f:
        config = yaml.safe_load(f)
    
    # Initialize W&B run
    wandb.init(
        project=config['wandb']['project'],
        name=config['wandb']['name'],
        entity=config['wandb']['entity'],
        tags=config['wandb']['tags'],
        config=config,
        resume=True  # Enable run resumption
    )
    
    print("✅ W&B initialized successfully!")
    print(f"📊 Dashboard: https://wandb.ai/{config['wandb']['entity']}/{config['wandb']['project']}")
    
except Exception as e:
    print(f"❌ W&B initialization failed: {e}")
    raise  # Re-raise to stop execution


In [ ]:
## 2. Training Preparation

### 2.1 Import Source Modules
Import our reusable source modules, consistent with how they're used in production scripts.


In [ ]:
# Add src to path for module imports (same as scripts)
import sys
import os
sys.path.append('/content/pokedex/src')  # For Colab

try:
    # Import reusable source modules (same as scripts)
    from training.yolo.trainer import YOLOTrainer
    from evaluation.yolo.evaluator import YOLOEvaluator
    
    print("✅ Source modules imported successfully!")
    print("Using same modular architecture as production scripts.")
    
except Exception as e:
    print(f"❌ Module import failed: {e}")
    raise  # Re-raise to stop execution


In [ ]:
### 2.2 Initialize Training
Set up the trainer with baseline configuration and check for existing checkpoints.


In [ ]:
# Initialize trainer with baseline configuration
try:
    # Use same configuration as production scripts
    config_path = "configs/yolov3/baseline_config.yaml"
    trainer = YOLOTrainer(config_path)
    
    # Check for existing checkpoints
    start_epoch = 0
    if os.path.exists(checkpoint_dir):
        checkpoint_files = sorted(os.listdir(checkpoint_dir))
        if checkpoint_files:
            latest_checkpoint = os.path.join(checkpoint_dir, checkpoint_files[-1])
            print(f"\n📦 Found checkpoint: {latest_checkpoint}")
            start_epoch = trainer.load_checkpoint()
            print(f"✅ Resuming from epoch {start_epoch}")
        else:
            print("\n📋 No existing checkpoints found. Starting fresh training.")
    
    print("\n🎯 Training Configuration:")
    print(f"• Config file: {config_path}")
    print(f"• Starting epoch: {start_epoch}")
    print(f"• Total epochs: {trainer.config['training']['epochs']}")
    print(f"• Batch size: {trainer.config['training']['batch_size']}")
    print(f"• Learning rate: {trainer.config['training']['learning_rate']}")
    print(f"• Checkpoint dir: {checkpoint_dir}")
    
except Exception as e:
    print(f"❌ Training initialization failed: {e}")
    raise  # Re-raise to stop execution


In [ ]:
## 3. Training Execution

### 3.1 Execute Training
Run the training process with proper error handling and progress monitoring.


In [ ]:
# Execute training with checkpoint support
print("🚀 Starting baseline training...")

try:
    # Start training from last checkpoint or beginning
    results = trainer.train(start_epoch=start_epoch)
    
    print("\n✅ Training completed successfully!")
    print("\n📊 Final Results:")
    for metric, value in results.items():
        print(f"• {metric}: {value:.4f}")
    
    # Save final model to Drive
    final_model_path = os.path.join(checkpoint_dir, "yolov3_final.pt")
    trainer.save_model(final_model_path)
    print(f"\n💾 Final model saved: {final_model_path}")
    
    # Log final artifacts to W&B
    wandb.save(final_model_path)
    print("\n📤 Model artifacts uploaded to W&B")
    
except KeyboardInterrupt:
    print("\n⚠️ Training interrupted by user!")
    print("Latest checkpoint was saved automatically.")
    print("You can resume training by running this notebook again.")
    
except Exception as e:
    print(f"\n❌ Training failed: {e}")
    raise  # Re-raise to stop execution


In [ ]:
### 3.2 Evaluate Model
Run evaluation on the test set to assess model performance.


In [ ]:
# Evaluate model on test set
try:
    # Initialize evaluator
    evaluator = YOLOEvaluator(trainer.model, trainer.config)
    
    # Run evaluation
    test_data = "liuhuanjim013/pokemon-yolo-1025"
    evaluation_results = evaluator.evaluate_model(test_data)
    
    print("\n📊 Model Evaluation Results:")
    for metric, value in evaluation_results.items():
        print(f"• {metric}: {value:.4f}")
    
    # Log evaluation results to W&B
    wandb.log({"final_evaluation": evaluation_results})
    
    print("\n📋 Original Blog Limitations Confirmed:")
    print("1. Poor performance in low light conditions")
    print("2. Sensitive to object size variations") 
    print("3. Background interference issues")
    print("4. Limited recognition accuracy")
    print("5. No advanced augmentation techniques")
    print("6. No learning rate scheduling")
    print("7. No early stopping mechanisms")
    
except Exception as e:
    print(f"\n❌ Evaluation failed: {e}")
    raise  # Re-raise to stop execution


In [ ]:
## 4. Cleanup & Next Steps

### 4.1 Resource Cleanup
Clean up resources and unmount Google Drive.


In [ ]:
# Clean up resources
try:
    # Finish W&B run
    wandb.finish()
    print("✅ W&B run completed and synced")
    
    # Unmount Google Drive
    from google.colab import drive
    drive.flush_and_unmount()
    print("✅ Google Drive unmounted safely")
    
    print("\n🎯 Next Steps:")
    print("1. Check W&B dashboard for training visualizations")
    print("2. Review saved checkpoints in Google Drive")
    print("3. Run improved_training.ipynb to address limitations")
    print("\n✨ Baseline reproduction completed successfully!")
    
except Exception as e:
    print(f"❌ Cleanup failed: {e}")
    print("⚠️ Please manually unmount Google Drive and close W&B run")


# YOLOv3 Baseline Training (Original Blog Reproduction)

This notebook reproduces the original blog's YOLOv3 training approach with 1025 classes.

**Goal**: Exact reproduction of original blog parameters and limitations.

**Dataset**: liuhuanjim013/pokemon-yolo-1025 (1025 classes)

**Original Blog**: https://www.cnblogs.com/xianmasamasa/p/18995912

**Environment Setup**: Uses centralized `setup_environment.py` for consistency with local development.


## 1. Environment Setup (Centralized)


In [ ]:
# Clone repository if not already available
# !git clone https://github.com/liuhuanjim013/pokedex.git
# %cd pokedex

# Use centralized environment setup script (consistent with local dev)
!python scripts/common/setup_environment.py --experiment yolo --colab

print("✅ Environment setup completed using centralized script!")
print("This ensures consistency with local development environment.")


## 2. Import Source Modules (Consistent with Scripts)


In [ ]:
# Add src to path for module imports (same as scripts)
import sys
import os
sys.path.append('/content/pokedex/src')  # For Colab
# sys.path.append(os.path.join(os.getcwd(), 'src'))  # For local development

# Import reusable source modules (same as scripts)
from training.yolo.trainer import YOLOTrainer

print("✅ Source modules imported successfully!")
print("Using same modular architecture as production scripts.")


## 3. Initialize Training (Same Logic as Scripts)


In [ ]:
# Use same configuration as production scripts
config_path = "configs/yolov3/baseline_config.yaml"

# Initialize trainer (same as train_yolov3_baseline.py)
trainer = YOLOTrainer(config_path)

print("🎯 YOLOv3 Baseline Training (Original Blog Reproduction)")
print(f"Configuration: {config_path}")
print("Goal: Reproduce original blog results exactly with 1025 classes")
print("\nOriginal blog parameters:")
print("- Epochs: 100")
print("- Batch size: 16") 
print("- Learning rate: 0.001")
print("- Minimal augmentation (only horizontal flip)")
print("- No learning rate scheduling")
print("- No early stopping")


## 4. Execute Training (Same as Scripts)


In [ ]:
# Execute training (identical logic to train_yolov3_baseline.py)
print("🚀 Starting baseline training...")

try:
    # Start training (same parameters as script)
    results = trainer.train()
    
    print("✅ Baseline training completed successfully!")
    print(f"Results: {results}")
    
except Exception as e:
    print(f"❌ Training failed: {e}")
    raise


## 5. Evaluation & Documentation


In [ ]:
# Evaluate model (same evaluation logic as scripts)
test_data = "liuhuanjim013/pokemon-yolo-1025"
evaluation_results = trainer.evaluate(test_data)

print("📊 Baseline Model Evaluation Results:")
for metric, value in evaluation_results.items():
    print(f"- {metric}: {value:.4f}")

print("\n📋 Original Blog Limitations Confirmed:")
print("1. Poor performance in low light conditions")
print("2. Sensitive to object size variations") 
print("3. Background interference issues")
print("4. Limited recognition accuracy")
print("5. No advanced augmentation techniques")
print("6. No learning rate scheduling")
print("7. No early stopping mechanisms")

print("\n✅ Baseline reproduction completed!")
print("Next step: Run improved_training.ipynb to address these limitations.")
